In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/resampled_bot-iot.csv')

print(df.head())


   pkSeqID         stime  flgs  proto  saddr    sport  daddr    dport  pkts  \
0        1  1.526344e+09     0      0    0.0      0.0    0.0      0.0     4   
1        2  1.526344e+09     0      1    0.0    139.0    0.0  36390.0    10   
2        3  1.526344e+09     0      2    0.0  51838.0    0.0    123.0     2   
3        4  1.526344e+09     0      0    0.0      0.0    0.0      0.0    10   
4        5  1.526344e+09     0      2    0.0  58999.0    0.0     53.0     4   

   bytes  ...  spkts  dpkts  sbytes  dbytes       rate     srate     drate  \
0    240  ...      2      2     120     120   0.002508  0.000836  0.000836   
1    680  ...      5      5     350     330   0.006190  0.002751  0.002751   
2    180  ...      1      1      90      90  20.590960  0.000000  0.000000   
3    510  ...      5      5     210     300   0.006189  0.002751  0.002751   
4    630  ...      2      2     174     456   0.005264  0.001755  0.001755   

   category  subcategory   attack  
0       0.0         

In [2]:
df = pd.read_csv('/content/resampled_bot-iot.csv', nrows=3997)

print(df.columns)



Index(['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
       'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
       'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
       'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'category', 'subcategory ', 'attack'],
      dtype='object')


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('/content/resampled_bot-iot.csv', nrows=3997)

# Select features and target
X = df[[
    'pkts',           # Total count of packets in transaction
    'bytes',          # Total number of bytes in transaction
    'dur',            # Total duration of transaction
    'mean',           # Average duration of aggregated records
    'stddev',         # Standard deviation of aggregated records
    'rate',           # Total packets per second in transaction
    'srate',          # Source-to-destination packets per second
    'drate',          # Destination-to-source packets per second
    'category',       # Traffic category
    'subcategory '    # Traffic subcategory
]]
y = df['attack']

# Initialize LabelEncoders for categorical features
label_encoders = {}
categorical_columns = ['category', 'subcategory ']

# Apply Label Encoding
for column in categorical_columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))  # Convert to string for consistent encoding
    label_encoders[column] = le

# Convert to numpy array
X = X.values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)


<ipython-input-3-9c06930e11ce>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))  # Convert to string for consistent encoding
<ipython-input-3-9c06930e11ce>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))  # Convert to string for consistent encoding


In [4]:
!pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-2.0.1-py3-none-any.whl size=193258 sha256=3395e451e2cb951980e14dead5638ada19060cc2ce5cec7ee8482a28f750a325
  Stored in directory: /root/.cache/pip/wheels/94/75/88/b853cf33b0053b0a001dca55b74d515048b7656e736364eb57
Successfully built pyod


In [5]:
from pyod.models.iforest import IForest
iforest_clf = IForest()
iforest_clf.fit(X_train)

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

iforest_pred = iforest_clf.predict(X_test)
print(classification_report(y_test, iforest_pred, digits=2))

              precision    recall  f1-score   support

           0       0.44      0.81      0.57       194
           1       0.21      0.05      0.08       206

    accuracy                           0.42       400
   macro avg       0.33      0.43      0.33       400
weighted avg       0.33      0.42      0.32       400

